In [1]:
from twitter_credentials import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from time import time
import csv
import os

In [2]:
sid = SentimentIntensityAnalyzer()

# twython

In [ ]:
from twython import TwythonStreamer

In [ ]:
class MyStreamer(TwythonStreamer):
    
    counter = 0
    limit = 0
#     data = []
    
    def on_success(self, data):
        if 'text' in data:
            if not self.limit == 0 and self.counter >= self.limit:
                self.disconnect()
                self.counter = 0
                return
            
            if 'retweeted_status' in data: # if found tweet is a retweet, return original tweet
                tweet = data['retweeted_status']
            else:
                tweet = data
                
#             print self.counter,': ', tweet['text']
#             print sid.polarity_scores(tweet['text'])
#             print

#             self.data.append(sid.polarity_scores(tweet['text'])['compound'])
            with open('data.csv', 'ab') as csvfile:
                spamwriter = csv.writer(csvfile, delimiter=' ',
                                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
                spamwriter.writerow([time(), sid.polarity_scores(tweet['text'])['compound']])
                
            self.counter += 1
            
            file_size = os.path.getsize('data.csv')
            if file_size > 1e9:
                self.disconnect()

    def on_error(self, status_code, data):
        print status_code

        # Want to stop trying to get data because of the error?
        # Uncomment the next line!
#         self.disconnect()

In [ ]:
stream = MyStreamer(twitter_credentials['API key'], twitter_credentials['API secret'],
                    twitter_credentials['token'], twitter_credentials['token secret'])
stream.limit = 0

In [ ]:
# stream.statuses.filter(track='cheese', language='en')
stream.statuses.sample(language='en')

In [ ]:
# remove zeros
# [value for value in MyStreamer.data if not value == 0]

In [ ]:
plt.hist(MyStreamer.data, 100)
plt.show()

In [ ]:
print np.mean(MyStreamer.data)
print np.std(MyStreamer.data)

<hr>
# tweepy

In [3]:
import tweepy

In [4]:
auth = tweepy.OAuthHandler(twitter_credentials['API key'], twitter_credentials['API secret'])
auth.set_access_token(twitter_credentials['token'], twitter_credentials['token secret'])

api = tweepy.API(auth)

In [5]:
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    
    limit = None
    counter = 0
    data_file = 'data.csv'
    data_limit = 1e9

    def on_status(self, status):
        
        global tweet
        tweet = status
        
        if hasattr(tweet, 'retweeted_status'):
            tweet = tweet.retweeted_status
        
        with open(self.data_file, 'ab') as csvfile:
                spamwriter = csv.writer(csvfile, delimiter=' ',
                                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
                spamwriter.writerow([time(), sid.polarity_scores(tweet.text)['compound']])
        
        self.counter += 1
        if self.limit and self.counter >= self.limit:
            return False
        file_size = os.path.getsize(self.data_file)
        if file_size > self.data_limit:
            return false
        
    def on_error(self, status_code):
        print status_code
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

In [13]:
myStreamListener = MyStreamListener()
myStreamListener.data_file = 'data_USA.csv'
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [16]:
myStream.disconnect()

In [ ]:
myStream.sample(languages=['en'])

In [15]:
myStream.filter(languages=['en'], locations=[-124.848974, 24.396308, -66.885444, 49.384358])

KeyboardInterrupt: 